In [1]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.1 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=160, type=sensor.other.collision), Actor(id=159, type=vehicle.mini.cooper_s_2021), Actor(id=146, type=traffic.traffic_light), Actor(id=145, type=traffic.traffic_light), Actor(id=144, type=traffic.traffic_light), Actor(id=143, type=traffic.traffic_light), Actor(id=142, type=traffic.traffic_light), Actor(id=141, type=traffic.traffic_light), Actor(id=140, type=traffic.traffic_light), Actor(id=139, type=traffic.traffic_light), Actor(id=138, type=traffic.traffic_light), Actor(id=137, type=traffic.traffic_light), Actor(id=136, type=traffic.traffic_light), Actor(id=135, type=traffic.traffic_light), Actor(id=134, type=traffic.traffic_light), Actor(id=133, type=traffic.traffic_light), Actor(id=132, type=traffic.traffic_light), Actor(id=131, type=traffic.traffic_light), Actor(id=130, type=traffic.traffic_light), Actor(id=129, type=traffic.traffic_light), Actor(id=128, type=traffic.traffic_light), Actor(id=127, type=traffic.traffic_light), Actor(id=126, type=traffic.traffic_light), Acto

In [3]:
manual_car = world.get_actor(159)
ego_vehicle = world.get_actor(159)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=-381.394470, y=-5.719271, z=0.003196)


In [4]:
def detect_surrounding_cars_outside_junction(
    key_value_pairs,
    junction_roads,
    lanes_all,
    ego_vehicle,
    world,
    distance_to_junc,
    junction,
):
    """
    Detects and records surrounding cars outside the junction in the city matrix.

    Parameters:
        key_value_pairs (list): A list of key-value pairs representing the city matrix.
        junction_roads (list): List of junction roads and lane information.
                               Format: [road_id, direction, outgoing_lane_wp, lane_id].
        lanes_all (dict): A dictionary containing lane information for different directions.
                          Format: {"ego": [lane_ids], "left": [lane_ids], "straight": [lane_ids], "right": [lane_ids]}.
        ego_vehicle (Vehicle): The ego vehicle.
        world (CarlaWorld): The world representation.
        distance_to_junc (float): Distance from ego vehicle to the junction in meters.
        junction (Junction): The junction object.

    Returns:
        list: Updated key-value pairs representing the city matrix with information about surrounding cars.
    """
    world_map = world.get_map()
    ego_waypoint = world_map.get_waypoint(ego_vehicle.get_location())
    junction_id = junction.id
    # junction_roads =
    #   [road_id of incoming road, direction from ego-perspective, wp of outgoing lane of incoming road, corresponding lane_id ],
    #       ... for all 4 directions ego, left, right, straight
    #   ]    # Special Highway Traffi

    surrounding_cars = {"ego": [], "left": [], "straight": [], "right": []}
    actors = world.get_actors()
    ego_actor = actors.find(ego_vehicle.id)
    actors = [ego_actor] + [actor for actor in actors if actor.id != ego_vehicle.id]
    if type(matrix) == dict:
        ego_already_in_matrix = any(1 in val for key, val in matrix.items()) # TODO: sometimes list, sometimes dict (whaaaat?)
    else:
        ego_already_in_matrix = any(1 in val for key, val in matrix)
    print(ego_already_in_matrix)
    for actor in actors:
        if "vehicle" in actor.type_id:# and actor.id != ego_vehicle.id:
            if actor.id == ego_vehicle.id and ego_already_in_matrix:
                continue
            actor_location = actor.get_location()
            actor_waypoint = world_map.get_waypoint(actor_location)
            if junction_id == 1368 and np.sign(ego_waypoint.lane_id) != np.sign(actor_waypoint.lane_id) and actor_waypoint.road_id in [40, 41]:
                continue
            for road in junction_roads:                
                distance_to_actor = distance(actor_location, road[2].transform.location)
                different_road_distance = None

                if (actor_waypoint.is_junction) and (
                    actor_waypoint.get_junction().id == junction_id
                ):
                    actor_outside_junction = False
                else:
                    actor_outside_junction = True

                if (
                    distance_to_actor <= distance_to_junc * 1.2
                ) and actor_outside_junction:  # add extra 20% because distance_to_actor is calculated with road[2] waypoint, which can be on other lane
                    included = True
                    if str(actor_waypoint.road_id) == road[0]:
                        different_road_distance = None
                    elif (
                        str(actor_waypoint.next(int(distance_to_junc / 2))[0].road_id)
                        == road[0]
                    ):
                        different_road_distance = int(distance_to_junc / 2)
                    elif (
                        str(
                            actor_waypoint.previous(int(distance_to_junc / 2))[
                                0
                            ].road_id
                        )
                        == road[0]
                    ) and (not actor_waypoint.is_junction):
                        different_road_distance = (
                            int(distance_to_junc / 2) * -1
                        )  # negative bc. look back from actor
                    elif (
                        str(actor_waypoint.next(int(distance_to_junc / 3))[0].road_id)
                        == road[0]
                    ):
                        different_road_distance = int(distance_to_junc / 3)
                    elif (
                        str(
                            actor_waypoint.previous(int(distance_to_junc / 3))[
                                0
                            ].road_id
                        )
                        == road[0]
                    ):
                        different_road_distance = (
                            int(distance_to_junc / 3) * -1
                        )  # negative bc. look back from actor
                    else:
                        included = False
                    if included:
                        surrounding_cars[road[1]].append(
                            (actor, different_road_distance)
                        )

    for road in junction_roads:
        for actor, different_road_distance in surrounding_cars[road[1]]:
            actor_location = actor.get_location()
            actor_waypoint = world_map.get_waypoint(actor_location)

            if not (different_road_distance is None):
                if different_road_distance > 0:
                    actor_waypoint_lane_id = actor_waypoint.next(
                        different_road_distance
                    )[0].lane_id
                elif different_road_distance < 0:
                    actor_waypoint_lane_id = actor_waypoint.previous(
                        (different_road_distance * -1)
                    )[0].lane_id
            else:
                actor_waypoint_lane_id = actor_waypoint.lane_id

            # sort list of lane ID's
            if (int(road[3]) < 0 and not junction.id == 1368) or int(road[3]) > 0 and junction.id == 1368:
                lanes_all[road[1]].sort(reverse=True)
            else:
                lanes_all[road[1]].sort()

            # determine lane of actor, i=0: lane most out of lanes incoming the junction
            for i in range(len(lanes_all[road[1]])):
                if lanes_all[road[1]][i] == actor_waypoint_lane_id:
                    break

            # determine number of cells/columns in matrix outside the inner junction
            if (road[1] == "ego") or (road[1] == "straight"):
                if len(lanes_all["left"]) == 0:
                    columns = int((8 - len(lanes_all["right"])) / 2)
                elif len(lanes_all["right"]) == 0:
                    columns = int((8 - len(lanes_all["left"])) / 2)
                else:
                    columns = int(
                        (8 - max([len(lanes_all["left"]), len(lanes_all["right"])])) / 2
                    )  # max number of lanes of right/left
            elif (road[1] == "left") or (road[1] == "right"):
                if len(lanes_all["ego"]) == 0:
                    columns = int((8 - len(lanes_all["straight"])) / 2)
                elif len(lanes_all["straight"]) == 0:
                    columns = int((8 - len(lanes_all["ego"])) / 2)
                else:
                    columns = int(
                        (8 - max([len(lanes_all["ego"]), len(lanes_all["straight"])]))
                        / 2
                    )  # max number of lanes of ego/straight

            # determine cell of actor in matrix
            if ((int(road[3]) > 0) and (actor_waypoint_lane_id > 0)) or (
                (int(road[3]) < 0) and (actor_waypoint_lane_id < 0)
            ):  # actor on outgoing lane
                # location.distance(waypoints[i].transform.location
                actor_distance_junction = actor_waypoint.transform.location.distance(
                    road[2].transform.location
                )
                if actor_distance_junction < (
                    distance_to_junc / columns
                ):  # doesn't matter how many columns: write "2" in the cell clostest to junction inner part
                    c = columns - 1
                elif (columns == 2) or (
                    actor_distance_junction < (distance_to_junc / columns) * 2
                ):  # if 2 columns then it must be the cell farthest away && if 3 columns check if junction is closer than 2/3 of DISTANCE_TO_JUNCTION, then middle cell
                    c = columns - 2
                else:  # if 3 columns and further away, then cell farthest away
                    c = columns - 3
            else:  # incoming actor
                if actor_waypoint.next(int(distance_to_junc / columns))[
                    0
                ].is_junction:  # doesn't matter how many columns: write "2" in the cell clostest to junction inner part
                    c = columns - 1
                elif (columns == 2) or (
                    actor_waypoint.next(int(distance_to_junc / columns) * 2)[
                        0
                    ].is_junction
                ):  # if 2 columns that it must be the cell farsest away && if 3 columns check if junction is closer than 2/3 of DISTANCE_TO_JUNCTION, then middle cell
                    c = columns - 2
                else:  # if 3 columns and further away than 10m, then cell farsest away
                    c = columns - 3

            # write "2" in identified cell
            if actor.id == ego_vehicle.id:
                cell_val = 1
            else:
                cell_val = 2
            if road[1] == "ego":
                for j in range(8):
                    # ego road
                    if key_value_pairs[-1 - j][1][c] != 3:
                        # in case ego vehicle is already in that matrix cell
                        if key_value_pairs[-1 - j - i][1][c] == 1:
                            # Calculate dot_product (similarity between the vectors)
                            dot_product = check_car_in_front_or_behind(
                                ego_vehicle.get_location(),
                                actor.get_location(),
                                ego_vehicle.get_transform().rotation,
                            )
                            if dot_product > 0:  # other car infront
                                if (columns != 1) and (
                                    c == columns - 1
                                ):  # already in front cell
                                    key_value_pairs[-1 - j - i][1][c - 1] = 1
                                    key_value_pairs[-1 - j - i][1][c] = cell_val
                                elif (
                                    columns != 1
                                ):  # not in front cell but more than 1 cells available
                                    key_value_pairs[-1 - j - i][1][c + 1] = 2
                            else:  # other car behind
                                if (columns != 1) and (c == 0):  # already in last cell
                                    key_value_pairs[-1 - j - i][1][c + 1] = 1
                                    key_value_pairs[-1 - j - i][1][c] = cell_val
                                elif (
                                    columns != 1
                                ):  # not in last cell but more than 1 cells available
                                    key_value_pairs[-1 - j - i][1][c - 1] = cell_val
                        else:
                            key_value_pairs[-1 - j - i][1][c] = cell_val
                        break
            elif road[1] == "left":
                for j in range(8):
                    # left road
                    if key_value_pairs[c][1][j] != 3:
                        key_value_pairs[c][1][j + i] = cell_val
                        break
            elif road[1] == "straight":
                for j in range(8):
                    # straight road
                    if key_value_pairs[j][1][-1 - c] != 3:
                        key_value_pairs[j + i][1][-1 - c] = cell_val
                        break
            elif road[1] == "right":
                for j in range(8):
                    # right road
                    if key_value_pairs[-1 - c][1][-1 - j] != 3:
                        key_value_pairs[-1 - c][1][-1 - j - i] = cell_val
                        break

    if junction.id == 1368:
        key_value_pairs[2][1][-3:], key_value_pairs[5][1][-3:] = key_value_pairs[5][1][-3:], key_value_pairs[2][1][-3:]
        key_value_pairs[3][1][-3:], key_value_pairs[4][1][-3:] = key_value_pairs[4][1][-3:], key_value_pairs[3][1][-3:]

    return key_value_pairs

In [5]:
def detect_ego_before_junction(
    key_value_pairs, junction_roads, lanes_all, lane_id_ego, ego_wp, distance_to_junc, junction
):
    """
    Update the city matrix to detect the position of the ego vehicle relative to a junction.

    Parameters:
        key_value_pairs (list of tuples): A list of tuples representing the city matrix.
        junction_roads (list of tuples): A list of tuples containing road IDs and lane IDs of the junction.
        lanes_all (dict): A dictionary containing lane IDs of the roads around the junction.
        lane_id_ego (int): The lane ID of the ego vehicle.
        ego_wp (carla.Waypoint): The waypoint of the ego vehicle.
        distance_to_junc (int): The distance in meters from the ego vehicle to the junction.

    Returns:
        list of tuples: The updated city matrix with the position of the ego vehicle detected.
    """
    # detect ego before junction


    # cast lane id's to int
    for road in junction_roads:
        lanes_all[road[1]] = [int(id) for id in lanes_all[road[1]]]

    print(lanes_all)

    # check on which lane ego is: i=0 means most out
    if lane_id_ego > 0:
        lanes_all["ego"].sort(reverse=True)
    else:
        lanes_all["ego"].sort()
    for i in range(int(len(lanes_all["ego"]))):
        if lanes_all["ego"][i] == lane_id_ego:
            break  # i=0: lane most out
    
    # check how many lanes the roads right and left of ego have
    if len(lanes_all["left"]) == 0:
        columns = int((8 - len(lanes_all["right"])) / 2)
    elif len(lanes_all["right"]) == 0:
        columns = int((8 - len(lanes_all["left"])) / 2)
    else:
        columns = int(
            (8 - min([len(lanes_all["left"]), len(lanes_all["right"])])) / 2
        )  # min number of lanes of right/left

    # determine cell of ego in matrix
    if ego_wp.next(int(distance_to_junc / columns))[
        0
    ].is_junction:  # doesn't matter how many columns: write "1" in the cell clostest to junction inner part
        c = columns - 1
    elif columns == 2:  # if 2 columns that it must be the cell farsest away
        c = columns - 2
    elif ego_wp.next(int(distance_to_junc / columns) * 2)[
        0
    ].is_junction:  # if 3 columns check if junction is closer than 10m, then middle cell
        c = columns - 2
    else:  # if 3 columns and further away than 10m, then cell farsest away
        c = columns - 3
    if ego_wp.road_id == 23:
        return key_value_pairs

    for j in range(8):
        if key_value_pairs[-1 - j][1][c] != 3:
            key_value_pairs[-1 - j - i][1][c] = 1
            break

    return key_value_pairs

In [6]:
junction = None
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
ego_on_bad_highway_street = False
highway_junction = False
highway_forward_vector = None
df = initialize_dataframe()
#try:
while True:
    print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    if not ego_on_highway:
        left_location = get_wp_x_units_to_left(ego_vehicle, 3)
        ego_on_highway = check_ego_on_highway(left_location, road_lane_ids, world_map)
        if ego_on_highway:
            ego_on_bad_highway_street = True
    else:
        ego_on_bad_highway_street = False
    if ego_on_highway:
        distance_to_junc = 80
    else:
        distance_to_junc = 20
    
    if junctions_detected and not ego_waypoint.is_junction:
        junctions_detected = False 
        same_junction = False
    elif junctions_detected:
        pass
    elif ego_waypoint.is_junction and on_junction == False:
        on_junction = True
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        junctions_detected = True
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction:
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_id = junction.id
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)


    # CASES
    # 1. On city junction
    if (ego_waypoint.is_junction and not highway_junction and junction and junction.id != 459) or (junction and junction.id == 1368 and ego_waypoint.is_junction):
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, wps, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif (is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction and junction.id != 459) or (junction and junction.id == 1368 and is_junction_ahead(ego_waypoint, distance_to_junc)):
        street_type = "Junction ahead"
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        wps = junction.get_waypoints(carla.LaneType().Driving)
        if junction.id == 1368: # traffic light junction logic
            if ego_on_highway: # already on highway traffic light junction logic
                street_type = "Highway traffic light"
                starting_wp = get_clostest_starting_waypoint(wps, ego_location)
                wps = remove_wps(starting_wp, wps)
                intial_wp = ego_waypoint
            else: # entrying highway traffic light jucntion
                starting_wp, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
                wps = remove_wps(starting_wp, wps)
                intial_wp = starting_wp.previous(5)[0]
        else: # Every other junction
            street_type = "Junction ahead"
            intial_wp = ego_waypoint

        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, intial_wp, wps, road_lane_ids, direction_angle, world_map
        )

        if junction_shape != "Error":
            print("Junction ahead")
            
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
                junction
            )
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, wps, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")


    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id and not check_ego_exit_highway(ego_vehicle, highway_forward_vector, highway_shape):
        street_type = "Highway with entry/exit"
        if distance(ego_location, junction.bounding_box.location) < 30 and not forward_vector_set:
            highway_forward_vector = ego_vehicle.get_transform().get_forward_vector()
            highway_forward_location = ego_vehicle.get_location()
            forward_vector_set = True
            exit_over = False
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            forward_vector_set = False
            same_junction = True
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
            highway_shape = get_highway_shape(wps, junction, ego_waypoint)
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old
            wrong_shape = True
        if ego_on_bad_highway_street:
            matrix = create_city_matrix(left_location, road_lane_ids, world_map, False, ego_on_bad_highway_street)
        else:
            matrix = create_city_matrix(ego_location, road_lane_ids, world_map, False, ego_on_bad_highway_street)                
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, direction_angle) 
                if not junction_old is None and junction_old.id != junction.id:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit, direction_angle, is_2nd_update=True) 
                
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")

    # 4. ego exiting Highway
    elif check_ego_exit_highway(ego_vehicle, highway_forward_vector, highway_shape) and not exit_over:
        print("On highway exit")
        street_type = "On highway exit"
        if highway_forward_vector and distance(highway_forward_location, ego_location) > 60:
            print("reset")
            highway_forward_vector = None
            highway_forward_location = None
        
        junction = get_junction_ahead(ego_waypoint, 1)
        if not junction:
            exit_over = True
            continue
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction, ego_waypoint)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, True, False) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")


    # 5. Highway entry
    elif is_junction_ahead(ego_waypoint, 20) and highway_junction and street_type in  ["Non highway street", "On junction", "On highway entry"]:

        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction, ego_waypoint)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, ghost=True, on_entry=on_entry) 

        for key, value in matrix.items():
            print(value)
        print("----------")

    # 6. Normal Road
    else:
        highway_forward_vector = None
        highway_forward_location = None
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)

    

    safe_data(ego_vehicle, matrix, street_type)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()
    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
    vehicles_list = [ego_vehicle]
    print("\ndestroying %d vehicles" % len(vehicles_list))
    client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

    time.sleep(0.5)
"""

----------------------------------
Junction ahead
{'ego': [6, 4, 5, 3], 'left': [-2, -3], 'straight': [5, 6, 3, 4], 'right': []}
False
1 [3, 3, 3, 0, 0, 3, 3, 3]
2 [3, 3, 3, 0, 1, 3, 3, 3]
3 [0, 0, 0, 0, 2, 0, 0, 0]
4 [0, 0, 0, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 0, 0]
7 [3, 3, 3, 3, 3, 3, 3, 3]
8 [3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------
Junction ahead
{'ego': [6, 4, 5, 3], 'left': [-2, -3], 'straight': [5, 6, 3, 4], 'right': []}
False
1 [3, 3, 3, 0, 0, 3, 3, 3]
2 [3, 3, 3, 0, 1, 3, 3, 3]
3 [0, 0, 0, 0, 2, 0, 0, 0]
4 [0, 0, 0, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 0, 0]
7 [3, 3, 3, 3, 3, 3, 3, 3]
8 [3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------
Junction ahead
{'ego': [6, 4, 5, 3], 'left': [-2, -3], 'straight': [5, 6, 3, 4], 'right': []}
False
1 [3, 3, 3, 0, 0, 3, 3, 3]
2 [3, 3, 3, 0, 1, 3, 3, 3]
3 [0, 0, 0, 0, 2, 0, 0, 0]
4 [0, 0, 0, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 0, 0]
7

KeyboardInterrupt: 

In [14]:
print(matrix)

{'1': [3, 3, 3, 0, 0, 3, 3, 3], '2': [3, 3, 3, 0, 0, 3, 3, 3], '3': [3, 3, 3, 0, 0, 3, 3, 3], '4': [0, 0, 0, 0, 0, 0, 0, 0], '5': [2, 0, 1, 0, 0, 0, 0, 0], '6': [3, 3, 3, 0, 0, 3, 3, 3], '7': [3, 3, 3, 0, 0, 3, 3, 3], '8': [3, 3, 3, 0, 0, 3, 3, 3]}


In [7]:
print(matrix)

{'1': [3, 3, 3, 0, 0, 3, 3, 3], '2': [3, 3, 3, 0, 0, 3, 3, 3], '3': [3, 3, 3, 0, 0, 3, 3, 3], '4': [0, 0, 0, 0, 0, 0, 0, 0], '5': [2, 1, 1, 0, 0, 0, 0, 0], '6': [3, 3, 3, 0, 0, 3, 3, 3], '7': [3, 3, 3, 0, 0, 3, 3, 3], '8': [3, 3, 3, 0, 0, 3, 3, 3]}


In [9]:
ego_already_in_matrix = any(1 in val for key, val in matrix.items())
ego_already_in_matrix

True

In [9]:
junction is not None

False

In [ ]:
# ToDo
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) config schreiben
 # 3) Doc strings (später)
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) exits
 # 2) Die ekligen junctions: Tankstelle und Highway Ampel (JP)
        # Einfach 4 Spure ballern
        # simpel wie anderen 
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) Highway Exit